# A Review of [WILDS: A Benchmark of in-the-Wild Distribution Shifts](https://arxiv.org/pdf/2012.07421.pdf)

## Problem statement
> What is the problem the paper aims to solve?

## Context/scope
> Why is this problem important or interesting?

## Existing work
> What has been done in literature?

## Contribution
> What is gap in literature that the paper is trying to fill? What is the unique contribution?

## Technical content (high level)
> What are the high level ideas behind their technical contribution?

## Technical content (details)
> Highlight (not copy and paste entire sections) the relevant details that are important to focus on (e.g. if there's a model, define it; if there is a theorem, state it and explain why it's important, etc).

## Experiments
> Which types of experiments were performed? What claims were these experiments trying to prove? Did the results prove the claims?

## Evaluation (your opinion)
> Do you think the work is technically sound? Do you think the proposed model/inference method is practical to use on real data and tasks? Do you think the experimental section was strong (there are sufficient evidence to support the claims and eliminate confounding factors)?

## Future work (for those interested in continuing research in a related field)
> Do you think you can suggest a concrete change or modification that would improve the existing solution(s) to the problem of interest? Try to implement some of these changes/modifications.

## Broader Impact
> How does this work potentially impact (both positively and negatively) the broader machine learning community and society at large when this technology is deployed? In the applications of this technology, who are the potentially human stakeholders? What are the potential risks to the interest of these stakeholders in the failure modes of this technology? Is there potential to exploit this technology for malicious purposes?

## Code
* At least one clear working pedagogical example demonstrating the problem the paper is claiming to solve. 
* At lease a bare bones implementation of the model/algorithm/solution (in some cases, you may be able to make assumptions  to simplify the model/algorithm/solution with the approval of your instructor)
* Demonstration on at least one instance that your implementation solves the problem.
* Demonstration on at least one instance the failure mode of the model/algorithm/solution, with an explanation for why failure occurred (is the dataset too large? Did you choose a bad hyper parameter?). The point of this is to point out edge cases to the user.

**NOTE**: Look at https://github.com/onefishy/am207_fall19_projects/blob/master/calibrated_uncertainty/calibrated_uncertainty_2/report.ipynb for inspiration regarding how they weaved code into the above sections.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=80cb1556-c762-4c6a-89b8-6410c844af03' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>